In [ ]:
!pip install safetensors huggingface_hub safetensors transformers accelerate

In [ ]:
!huggingface-cli login

In [ ]:
!git config --global credential.helper store

In [ ]:
import torch
from safetensors import safe_open
from safetensors.torch import save_file
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dfurman/llama-7b-fp16")

model = AutoModelForCausalLM.from_pretrained("dfurman/llama-7b-fp16",
                                             torch_dtype=torch.float16)
model

In [ ]:
!mkdir llama_7b
model.save_pretrained('/content/llama_7b')
tokenizer.save_pretrained('/content/llama_7b')

In [ ]:
import glob
glob.glob('/content/llama_7b/*.bin')

In [ ]:
# iterate over each bin file in model folder
# expected filenames: pytorch_model-0000x-of-0000y.bin

bins = glob.glob('/content/llama_7b/*.bin')

for tensor_path in bins:
    tensor = torch.load(tensor_path)
    new_name = tensor_path.split('/')[-1].split('pytorch_')[-1].split('.bin')[0]
    print(tensor_path)
    print(new_name)
    save_file(tensor, f"/content/llama_7b/{new_name}.safetensors")


    tensors = {}
    with safe_open(f"/content/llama_7b/{new_name}.safetensors", framework="pt", device="cpu") as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)
    print(tensors.keys())
    print("\n")


In [ ]:
model_name = "/content/llama_7b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)

print(model)


In [ ]:

inputs = tokenizer("Paris is the capital of", return_tensors="pt", return_token_type_ids=False)

# Example 1: Print the scores for each token generated with Greedy Search
outputs = model.generate(**inputs, max_new_tokens=100)
tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# optionally push to hub

model.push_to_hub("dfurman/llama-7b-fp16", use_auth_token=True, safe_serialization=True)
tokenizer.push_to_hub("dfurman/llama-7b-fp16", use_auth_token=True)